In [ ]:
import numpy as np
np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

# TP2 - Sélection de variables semi-supervisée

## Dataset: Wave.txt (vagues de Brieman)
- 5000 individus
- 40 variables  
- 3 classes

## I. Découpage de la base en apprentissage/test

In [ ]:
# Lecture du fichier Wave.txt
df = pd.read_csv('C:\\Users\\hp\\OneDrive\\Documents\\POLYTECH\\2025_2026\\Big_data_tp1\\Wave.txt', sep='\s+', header=None, engine='python')
print("Loaded df shape:", df.shape)

# Renommer les colonnes
n_cols = df.shape[1]
feature_names = [f'X{i}' for i in range(n_cols - 1)] + ['label']
df.columns = feature_names

print("\nAperçu des données:")
print(df.head())

# Vérification des labels
y = df['label']
print("\nInformations sur les labels:")
print(f"  - Nombre de classes: {y.nunique()}")
print(f"  - Distribution:\n{y.value_counts().sort_index()}")

In [ ]:
def stratified_split(df, test_size=0.5, random_state=42):
    """Découpe stratifiée ET normalisation automatique."""
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Découpage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Normalisation
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    A = pd.DataFrame(X_train_scaled, columns=X.columns)
    A['label'] = y_train.values
    
    T = pd.DataFrame(X_test_scaled, columns=X.columns)
    T['label'] = y_test.values

    print("Données normalisées avec succès (Moyenne=0, Écart-type=1)")
    return A, T

In [ ]:
A , T = stratified_split(df, test_size=0.5, random_state=42)

## II. Simulation de l'aspect semi-supervisé

In [ ]:
def semi_label(A, pct=0.1, random_state=42):
    np.random.seed(random_state)
    A_copy = A.copy().reset_index(drop=True)
    
    # Identification des colonnes
    y_col = A_copy.columns[-1]   # La colonne 'label'
    
    # Sélection des indices à étiqueter
    n_samples = A_copy.shape[0]
    n_labeled = int(n_samples * pct)
    
    indices = np.arange(n_samples)
    labeled_indices = np.random.choice(indices, n_labeled, replace=False)
    unlabeled_indices = np.setdiff1d(indices, labeled_indices)

    # Création des deux sous-ensembles
    A_etiq = A_copy.iloc[labeled_indices].copy()
    A_non_etiq = A_copy.iloc[unlabeled_indices].copy()
    
    # On "cache" les labels de la partie non-étiquetée pour la simulation
    A_non_etiq[y_col] = np.nan

    print(f"Simulation semi-supervisée prête (Données déjà normalisées).")
    print(f" - Échantillons étiquetés: {len(A_etiq)}")
    print(f" - Échantillons non étiquetés: {len(A_non_etiq)}")
    
    return A_etiq, A_non_etiq

In [ ]:
A_etiq, A_non_etiq = semi_label(A, pct=0.1, random_state=42)

In [ ]:
def S1(x, y):
    mu = x.mean()
    between_var = 0.0
    within_var = 0.0
    
    for label, group in x.groupby(y):
        n_k = len(group)
        mu_k = group.mean()
        
        sigma_k_sq = np.var(group) 
        
        between_var += n_k * (mu_k - mu) ** 2
        within_var += n_k * sigma_k_sq
 
    
    return between_var / within_var

In [ ]:
def S2(x, t=10):
    x_arr = x.values.astype(float)
    var = np.var(x_arr)
    
    if var == 0 or np.isnan(var):
        return np.nan
    
    n = len(x_arr)
    
    diff_matrix = x_arr[:, None] - x_arr[None, :]  

    S_matrix = np.exp(-(diff_matrix ** 2) / t)
    
    numerator = np.sum((diff_matrix ** 2) * S_matrix)
    score = numerator / var
    
    return score

In [ ]:
from scipy.spatial.distance import pdist, squareform

def compute_scores(A_etiq, A_non_etiq, t=10):
   
    X_etiq = A_etiq.drop(columns=["label"])
    y = A_etiq["label"]
    X_non = A_non_etiq.drop(columns=["label"])
   
    dist_sq = squareform(pdist(X_non, 'sqeuclidean'))
    S_matrix = np.exp(-dist_sq / t)

    scores = {}
    s1_dict = {}
    s2_dict = {}
    
    print("Calcul des scores pour chaque variable...")
    for i, v in enumerate(X_etiq.columns):
        if (i + 1) % 10 == 0:
            print(f"  Progression: {i + 1}/{len(X_etiq.columns)}")
        
        # S1 (Fisher)
        s1 = S1(X_etiq[v], y)
        
        # S2 (Laplace) 
        x_v = X_non[v].values
        var_v = np.var(x_v)
        
        diff_v = (x_v[:, None] - x_v[None, :]) ** 2
        numerator = np.sum(diff_v * S_matrix)
        s2 = numerator / var_v
        
        s1_dict[v] = s1
        s2_dict[v] = s2
        
        # Ratio final
        if s2 > 0 and not np.isinf(s2):
            scores[v] = s1 / s2
        else:
            scores[v] = 0.0
            
    return pd.Series(scores), s1_dict, s2_dict


# On récupère bien les 3 dictionnaires renvoyés par la fonction
scores, s1_results, s2_results = compute_scores(A_etiq, A_non_etiq, t=10)
print(scores)


## IV. Évaluation de la sélection



In [ ]:
# Histogramme des pertinences
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
scores_clean = scores.dropna()
plt.bar(range(len(scores_clean)), scores_clean.values, color='steelblue', alpha=0.7)
plt.xlabel('Variables (triées par score)', fontsize=11)
plt.ylabel('Score S1/S2', fontsize=11)
plt.title('Histogramme des pertinences de toutes les variables', fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
def evaluate_mlp(A_etiq, T, selected_vars, normalize=False, verbose=True):
    
    # Préparation des données
    X_train = A_etiq[selected_vars]
    y_train = A_etiq['label']
    X_test = T[selected_vars]
    y_test = T['label']
    
    
    if normalize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    # Entraînement du MLP
    mlp = MLPClassifier(hidden_layer_sizes=(12, 10, 10), max_iter=5000, random_state=42)
    
    mlp.fit(X_train, y_train)
    
    # Prédiction
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    if verbose:
        print(f"  Nombre de variables: {len(selected_vars)}")
        print(f"  Normalisation: {'Oui' if normalize else 'Non'}")
        print(f"  Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    return accuracy

In [ ]:
# Courbe d'efficacité avec/sans normalisation 

n_vars_range = list(range(5, 41, 5))  
acc_no_norm = []
acc_with_norm = []

for n in n_vars_range:
    selected = scores.head(n).index.tolist()
    
    print(f"\nTest avec {n} variables:")
    print("  Sans normalisation:")
    acc_nn = evaluate_mlp(A_etiq, T, selected, normalize=False)
    acc_no_norm.append(acc_nn)
    
    print("  Avec normalisation:")
    acc_wn = evaluate_mlp(A_etiq, T, selected, normalize=True)
    acc_with_norm.append(acc_wn)

# Graphique
plt.figure(figsize=(10, 6))
plt.plot(n_vars_range, acc_no_norm, 'o-', label='Sans normalisation', linewidth=2, markersize=8)
plt.plot(n_vars_range, acc_with_norm, 's-', label='Avec normalisation', linewidth=2, markersize=8)
plt.xlabel('Nombre de variables sélectionnées', fontsize=12)
plt.ylabel('Accuracy sur T', fontsize=12)
plt.title('Courbe d\'efficacité du MLP (par tranches de 5 variables)', fontweight='bold', fontsize=13)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

La précision augmente de 49% à 80% entre 5 et 15 variables, ce qui prouve que le classement identifie les variables les plus riches en signal dès le début.
Le pic de performance est atteint à 20 variables (83%). C'est le point d'équilibre où le modèle a assez d'informations sans être pollué par le bruit.
Au-delà de 20 variables, l'accuracy chute ou stagne (78% à la fin), ce qui confirme que les variables à faible score sont inutiles, voire nuisibles.
Avec seulement 5% de données étiquetées, on obtient déjà une précision de 83%, ce qui montre que la structure globale des données est capturée même avec très peu de labels.On constate également que la normalisation stabilise l'apprentissage et empêche l'effondrement de la précision quand le nombre de variables devient trop élevé.

In [ ]:
# Courbe sur variables non-pertinentes (normalisation activée)

print("ÉVALUATION: VARIABLES NON-PERTINENTES")

scores_reversed = scores.sort_values(ascending=True)

acc_non_pertinent = []
for n in n_vars_range:
    selected = scores_reversed.head(n).index.tolist()
    
    print(f"\nTest avec {n} variables NON-pertinentes:")
    acc = evaluate_mlp(A_etiq, T, selected, normalize=True)
    acc_non_pertinent.append(acc)


plt.figure(figsize=(10, 6))
plt.plot(n_vars_range, acc_with_norm, 's-', label='Variables pertinentes (top)', 
         linewidth=2, markersize=8, color='forestgreen')
plt.plot(n_vars_range, acc_non_pertinent, '^-', label='Variables non-pertinentes (bottom)', 
         linewidth=2, markersize=8, color='crimson')
plt.xlabel('Nombre de variables sélectionnées', fontsize=12)
plt.ylabel('Accuracy sur T', fontsize=12)
plt.title('Comparaison: Variables pertinentes vs non-pertinentes (avec normalisation)', 
          fontweight='bold', fontsize=13)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Préparation des données manquantes
all_features = scores.index.tolist()

# Courbe "Sans sélection" (constante sur l'axe X pour comparaison)
acc_full_norm = evaluate_mlp(A_etiq, T, all_features, normalize=True, verbose=False)
acc_full_no_norm = evaluate_mlp(A_etiq, T, all_features, normalize=False, verbose=False)


plt.figure(figsize=(12, 7))

# 1. Variables pertinentes (Normalisées)
plt.plot(n_vars_range, acc_with_norm, 's-', label='Variables pertinentes (normalisées)', 
         color='tab:blue', linewidth=2, markersize=8)

# 2. Variables pertinentes (Non normalisées)
plt.plot(n_vars_range, acc_no_norm, 'o-', label='Variables pertinentes (non normalisées)', 
         color='tab:orange', linewidth=2, markersize=8)

# 3. Variables NON-pertinentes (Normalisées)
plt.plot(n_vars_range, acc_non_pertinent, '^-', label='Variables non-pertinentes', 
         color='tab:red', linewidth=2, markersize=8)

# 4. Sans sélection de variables (Normalisées) 
plt.axhline(y=acc_full_norm, color='tab:blue', linestyle='--', alpha=0.6, 
            label=f'Sans sélection (norm) : {acc_full_norm:.4f}')

# 5. Sans sélection de variables (Non normalisées)
plt.axhline(y=acc_full_no_norm, color='tab:orange', linestyle='--', alpha=0.6, 
            label=f'Sans sélection (non-norm) : {acc_full_no_norm:.4f}')

# Mise en forme
plt.title("Courbes d'efficacité du MLP selon la sélection de variables", fontweight='bold', fontsize=14)
plt.xlabel("Nombre de variables sélectionnées", fontsize=12)
plt.ylabel("Précision (Accuracy) sur T", fontsize=12)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, linestyle='-', alpha=0.3)
plt.xticks(n_vars_range)

plt.tight_layout()
plt.show()

In [ ]:
# Courbe selon la sélection de variables (normalisation activée)
print("ÉVALUATION: IMPACT DU NOMBRE DE VARIABLES")


# Test plus fin: de 1 à 40 variables
n_vars_detailed = list(range(1, 41))
acc_detailed = []

for n in n_vars_detailed:
    selected = scores.head(n).index.tolist()
    acc = evaluate_mlp(A_etiq, T, selected, normalize=True, verbose=False)
    acc_detailed.append(acc)
    
    if n % 10 == 0:
        print(f"  {n} variables: accuracy = {acc:.4f}")

# Graphique
plt.figure(figsize=(12, 6))
plt.plot(n_vars_detailed, acc_detailed, 'o-', linewidth=2, markersize=5, color='steelblue')
plt.axvline(x=20, color='red', linestyle='--', alpha=0.7, label='20 variables')
plt.xlabel('Nombre de variables sélectionnées', fontsize=12)
plt.ylabel('Accuracy sur T', fontsize=12)
plt.title('Courbe de performance selon la sélection de variables (avec normalisation)', 
          fontweight='bold', fontsize=13)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Trouver le meilleur nombre de variables
best_n = n_vars_detailed[np.argmax(acc_detailed)]
best_acc = max(acc_detailed)
print(f"\n Meilleur nombre de variables: {best_n} (accuracy = {best_acc:.4f})")

In [ ]:
# Impact du % de données labellisées (fixé à 20 variables)

print("ÉVALUATION: IMPACT DU % DE DONNÉES LABELLISÉES")


pct_range = [0.05, 0.10, 0.15, 0.20, 0.30, 0.40, 0.50, 0.75, 1.0]
acc_by_pct = []

for pct in pct_range:
    print(f"\nTest avec {pct*100:.0f}% de données labellisées:")
    
    A_etiq_temp, A_non_etiq_temp = semi_label(A, pct=pct, random_state=42)
    
    scores_temp, _, _ = compute_scores(A_etiq_temp, A_non_etiq_temp, t=10)
    
    selected_20 = scores_temp.head(20).index.tolist()
     
    acc = evaluate_mlp(A_etiq_temp, T, selected_20, normalize=True)
    acc_by_pct.append(acc)

# Graphique
plt.figure(figsize=(10, 6))
pct_labels = [f"{p*100:.0f}%" for p in pct_range]
plt.plot(pct_range, acc_by_pct, 'o-', linewidth=2, markersize=10, color='darkviolet')
plt.xlabel('% de données labellisées dans A', fontsize=12)
plt.ylabel('Accuracy sur T', fontsize=12)
plt.title('Courbe de performance en fonction du % de données labellisées\n(20 variables sélectionnées, avec normalisation)', 
          fontweight='bold', fontsize=13)
plt.xticks(pct_range, pct_labels)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMeilleur % de labellisation: {pct_range[np.argmax(acc_by_pct)]*100:.0f}% "
      f"(accuracy = {max(acc_by_pct):.4f})")

L'accuracy atteint son maximum lorsque le pourcentage de données labellisées est de 50% ainsi qu'à 100%. Cela montre que l'apprentissage semi-supervisé présente des avantages significatifs, pouvant même générer des résultats comparables à ceux obtenus avec 100% des données labellisées, c'est-à-dire dans un contexte d'apprentissage entièrement supervisé.